In [2]:
%pylab inline
import random
import re
import sys
import time
from multiprocessing import Process, Queue
import networkx as nx
import networkx.algorithms.centrality as nxcent
import networkx.algorithms.connectivity as nxconn
import networkx.algorithms.distance_measures as nxdist
import networkx.algorithms.components as nxcomp
import numpy as np
import elp_networks as elpnet
import elp_networks.algorithms as elpalg
import logbook

Populating the interactive namespace from numpy and matplotlib


In [3]:
net_file = "external/as20000102.csv"
exp_name = "simulate_router"

In [4]:
edge_list = []
whitespace = re.compile(r"\w+")
nodes = set()
with open(net_file, "rb") as f:
    for row in f:
        if row.startswith("#"):
            continue
        source, target = re.split(r"\W+", row.strip())
        source = int(source.strip())
        target = int(target.strip())
        nodes.add(source)
        nodes.add(target)
        edge_list.append( (source,target) )
node_count = len(nodes)

In [7]:
g.degree().items()

[(0, 378),
 (1, 1460),
 (2, 286),
 (3, 128),
 (4, 81),
 (5, 140),
 (6, 693),
 (7, 401),
 (8, 111),
 (9, 752),
 (10, 31),
 (11, 1),
 (12, 23),
 (13, 2),
 (14, 1),
 (15, 43),
 (16, 34),
 (17, 4),
 (18, 32),
 (19, 33),
 (20, 63),
 (21, 112),
 (22, 252),
 (23, 3),
 (24, 94),
 (25, 11),
 (26, 133),
 (27, 17),
 (28, 180),
 (29, 2),
 (30, 6),
 (31, 73),
 (32, 6),
 (33, 5),
 (34, 26),
 (35, 11),
 (36, 3),
 (37, 5),
 (38, 51),
 (39, 9),
 (40, 4),
 (41, 226),
 (42, 8),
 (43, 13),
 (44, 3),
 (45, 15),
 (46, 2),
 (47, 104),
 (48, 20),
 (49, 29),
 (50, 4),
 (51, 87),
 (52, 64),
 (53, 17),
 (54, 21),
 (55, 3),
 (56, 48),
 (57, 25),
 (58, 12),
 (59, 30),
 (60, 142),
 (61, 21),
 (62, 54),
 (63, 69),
 (64, 41),
 (65, 52),
 (66, 6),
 (67, 54),
 (68, 9),
 (69, 2),
 (70, 12),
 (71, 2),
 (72, 10),
 (73, 2),
 (74, 4),
 (75, 4),
 (76, 40),
 (77, 22),
 (78, 2),
 (79, 4),
 (80, 2),
 (81, 3),
 (82, 11),
 (83, 2),
 (84, 5),
 (85, 12),
 (86, 34),
 (87, 10),
 (88, 4),
 (89, 6),
 (90, 7),
 (91, 5),
 (92, 18),
 (93,

In [17]:
subnodes = [x[0] for x in sorted(g.degree().items(), key=lambda x: x[1], reverse=True)[:4608]]

In [18]:
subnodes

[1,
 9,
 6,
 7,
 0,
 2,
 22,
 41,
 28,
 1032,
 60,
 5,
 26,
 475,
 550,
 3,
 21,
 8,
 551,
 47,
 24,
 456,
 51,
 4,
 857,
 801,
 31,
 629,
 63,
 476,
 1112,
 52,
 20,
 381,
 3058,
 631,
 1418,
 2181,
 438,
 856,
 62,
 67,
 65,
 901,
 2610,
 38,
 248,
 995,
 2491,
 56,
 892,
 1092,
 1108,
 393,
 653,
 1558,
 180,
 15,
 64,
 600,
 868,
 76,
 1050,
 2194,
 423,
 876,
 2183,
 131,
 384,
 1121,
 1196,
 2202,
 2218,
 3796,
 16,
 86,
 902,
 19,
 2204,
 18,
 903,
 1285,
 3845,
 10,
 547,
 922,
 938,
 2407,
 59,
 536,
 579,
 843,
 2206,
 49,
 847,
 923,
 540,
 864,
 1118,
 3486,
 3646,
 34,
 394,
 469,
 820,
 1117,
 3506,
 3900,
 4547,
 57,
 101,
 129,
 899,
 437,
 562,
 904,
 1045,
 3780,
 12,
 848,
 2187,
 77,
 1295,
 3197,
 54,
 61,
 491,
 817,
 900,
 1403,
 3893,
 48,
 539,
 874,
 1346,
 2412,
 2635,
 4179,
 4334,
 4549,
 2196,
 2268,
 3062,
 3150,
 3435,
 92,
 100,
 117,
 173,
 552,
 2190,
 2199,
 2590,
 3533,
 3969,
 4573,
 5060,
 27,
 53,
 196,
 207,
 242,
 563,
 845,
 859,
 914,
 1046,


In [5]:
nodes = g.nodes()
num_pairs = 10
sources = random.sample(nodes, num_pairs)
targets = random.sample(nodes, num_pairs)
# Prevent self loops
for i in range(num_pairs - 1):
    if sources[i] == targets[i]:
        sources[i], sources[-1] = sources[-1], sources[i]
if sources[-1] == targets[-1]:
    sources[-1] = random.choice(set(nodes) - set(sources))

In [7]:
connectivities = []
start_ts = time.time()
for source, target in zip(sources, targets):
    c = nxconn.node_connectivity(g, source, target)
    connectivities.append(c)
print "Time per iteration: %f" % ((time.time() - start_ts) / float(len(connectivities)))

Time per iteration: 0.447809


In [13]:
connectivities

[1, 2, 2, 1, 1, 1, 2, 2, 1, 1]

In [1]:
g

NameError: name 'g' is not defined